In [1]:
import numpy as np
import matplotlib.pyplot as plt
from casadi import *
from casadi.tools import *
import pdb
import sys
import do_mpc

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time

from examples.batch_reactor.template_model import template_model
from examples.batch_reactor.template_mpc import template_mpc
from examples.batch_reactor.template_simulator import template_simulator

""" User settings: """
show_animation = False
store_results = False

"""
Get configured do-mpc modules:
"""

model = template_model()
mpc = template_mpc(model)
simulator = template_simulator(model)
estimator = do_mpc.estimator.StateFeedback(model)

"""
Set initial state
"""

X_s_0 = 1.0 # This is the initial concentration inside the tank [mol/l]
S_s_0 = 0.5 # This is the controlled variable [mol/l]
P_s_0 = 0.0 #[C]
V_s_0 = 120.0 #[C]
x0 = np.array([X_s_0, S_s_0, P_s_0, V_s_0])

mpc.x0 = x0
simulator.x0 = x0
estimator.x0 = x0

mpc.set_initial_guess()
for k in range(150):
    u0 = mpc.make_step(x0)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
    print(k, x0, u0)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2244
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      839

Total number of variables............................:      608
                     variables with only lower bounds:      268
                variables with lower and upper bounds:      288
                     variables with only upper bounds:        0
Total number of equa

In [5]:
estimator.x0.keys()



['X_s', 'S_s', 'P_s', 'V_s']

In [ ]:
import gym
class BatchReactorEnv(gym.Env):
        def __init__(self, 
                     model, 
                     simulator, 
                     estimator,
                     steady_observation=None):
                self.model = model
                self.simulator = simulator
                self.estimator = estimator
                self.action_dim = len(self.simulator.u0.keys())
                self.observation_dim = len(self.simulator.x0.keys())
                # each scalar shall implement "transform" and "inverse_transform"
                # as well min_values and max_values
                self.action_scalar = None
                self.reward_scalar = None
                self.observation_scalar = None
                self.steady_observation = steady_observation
                self.min_observation, self.max_observation = self.observation_scalar.min_values, self.observation_scalar.max_values
                self.min_actions, self.max_actions = self.action_scalar.min_values, self.action_scalar.max_values
                if self.model.model_type == "discrete":
                        self.action_space = gym.spaces.Discrete(self.action_dim)
                else:
                        self.action_space = gym.spaces.Box(low=self.min_actions, high=self.max_actions, shape=(self.action_dim,))
                self.observation_space = gym.spaces.Box(low=self.min_observation, high=self.max_observation, shape=(self.observation_dim,))
                self.cur_step = 0
                self.episode_len = 50
                
        def get_reward(self, state, action):
                return 0.0
        
        def step(self, action):
                u0 = self.action_scalar.inverse_transform(action)
                y_next = self.simulator.make_step(u0)
                x0 = self.estimator.make_step(y_next)
                state = self.observation_scalar.transform(x0)
                reward = self.get_reward(state, action)
                self.cur_step += 1
                done = (self.cur_step >= self.episode_len)
                info = {}
                return state, reward, done, info
        
        def reset(self, x0=np.array([1.0, 0.5, 0.0, 120.0])):
                self.simulator.reset_history()
                self.simulator.x0 = x0
                self.estimator.x0 = x0
                return x0